# DRAFT POST PROCESSING WORKFLOW
This notebook mocks up the front end user interactions with the seafloor geodesy code.  The final version of this will be a major deliverable to the USGS on or before Sept 31, 2024.

Requirements for Minimum Viable Product:
- Earthscope will deliver a suite of notebooks and a containerized environment for the USGS to run end-to-end GARPOS seafloor positioning in AWS.  The software must support the data preparation steps and processing of data from a GNSS-A survey performed using either an SV2 or SV3 system.  A survey is defined here as a set of observations that will be processed as one or more sessions and merged to generate a single position for either a single transponder or an array center.

In [7]:
from es_sfgtools.utils.gage_data import (
    generate_gage_data_survey_url, 
    get_survey_file_dict
    )

data_directory = '../data' 
token_path = '.' # directory where the token will be stored (default is the current directory)

## Step 1. Initial User Input to select Observation Data

#### 1.1 Browse available surveys from the community archive
- Locate the survey of interest in https://gage-data.earthscope.org/archive/seafloor, and note the `network`, `station`, and `survey` names
- In order to use this notebook to process new surveys, the data must first be submitted and made available from the community archive 

#### 1.2 Select a survey and view available files
- User inputs `network`, `station`, and `survey`
- Notebook locates and presents a summary available files for that survey
- You will be asked to log in with your EarthScope credentials during your first interaction with the archive server. The token will be refreshed every 8 hours.

In [8]:
network = 'aleutian'
station = 'SEM1'
survey = '2018_A_SFG1'

url = generate_gage_data_survey_url(network, station, survey)
survey_files = get_survey_file_dict(url)

Found:
    1 master file(s)
    163 bcnovatel file(s)
    163 bcsonardyne file(s)
    1 lever_arms file(s)
    2 ctd file(s)


## Step 2.  Prepare observation data.  
In order to support variable Session lengths, we will first build multiple DataFrames based on raw file length (1hr for SV2, variable for SV3), and then merge those into DataFrames for the entire survey period.  

#### 2.1 Acoustic Data Prep
- Loop through all raw acoustic files in survey, read in data to AcousticDataFrames, and then merge all those into a single AcousticDataFrame.
- Save to csv, implement logic to load from csv if exists already.

#### 2.2 IMU Data Prep
- Loop through all raw IMU files in survey, read in data to ImuDataFrames, and then merge all those into a single ImuDataFrame
- Save to csv, implement logic to load from csv if exists already.

#### 2.3 GNSS Novatel messages to Rinex
- Loop through each raw GNSS observable file, writing a RINEX file for each.  
- Present metadata to be used for RINEX headers, provide option to update if incorrect.
- If ShotDataFrame csv exists, can skip 2.3 - 2.6 and load from csv.

#### 2.4 GNSS Rinex to Kin
- Loop through each rinex files, run PRIDE-PPP, generate and store Kin files.  
- Expose parameters for PRIDE-PPP function pdp3 if user wants additional control over waveglider position processing.
- If ShotDataFrame csv exists, can skip 2.3 - 2.6 and load from csv.

#### 2.5 Build PositionDataFrame
- Read in the Kin files to PositionDataFrames, and merge those into a single PositionDataFrame.
- If ShotDataFrame csv exists, can skip 2.3 - 2.6 and load from csv.

#### 2.6 Build ShotDataFrame
- Combine the AcousticDataFrame, ImuDataFrame, and PositionDataFrame into a single ShotDataFrame, interpolating the IMU and Position data to match the 15s sample interval in the Acoustic Data.  
- Store this ShotDataFrame as csv or tab-delimited ascii.
- If ShotDataFrame csv exists, can skip 2.3 - 2.6 and load from csv.

## Step 3. Select and prepare site config data (CTD, lever arms, and master files)

#### 3.1 Load CTD
- Load .cnv file if present in the survey directory, present relevant metadata from header so user knows what CTD this is
- Allow user to specify different CTD if desired, or none present in directory  
- Parse CTD and build SoundVelocityProfile    
    
   


#### 3.2 Verify/select ATD Offset
- Load lever_arms file if present in the survey directory, display offset values for sanity check
- Allow user to specify different lever arms file if desired, or none present in directory

#### 3.3 Verify/select Master File
- Load master file if present in the survey directory, display relevant metadata for sanity check
- Allow user to specify different master file if desired, or none present in directory 
- Build GarposSite from lever_arms and master files

## Step 4. Run GARPOS Sessions

#### 4.1 Verify processing start and stop times
- visualize the waveglider position data
    - allow user to trim start/end times based on survey pattern, differentiate between circle drives and fixed point patterns within a given survey

#### 4.2 Enter GARPOS parameters
- Enter the desired garpos processing session length (ie 1 hr, 24 hrs)
- Show default hyper-parameters used for GARPOS processing, allow user to override
- Select whether computing array center or single transponder position  

#### 4.3 Run GARPOS 
- Slice the ShotDataFrame into chunks matching the Session length.   
- Process each session and store all results as csv / tab-delimeted ascii

#### 4.4 View residuals
- make a simple plot of GARPOS epoch by epoch residuals for the whole survey

## Step 5. Merge GARPOS Sessions

#### 5.1 Run session merge
- Use GARPOS to merge the session results into a single survey result.  
- Store results as csv/tab-delimited